Cart Pole source code: https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gym
import time
from tqdm import tqdm
import rl.features.tile_coding as tc

np.random.seed(42)

# Cart Pole environment

In [ ]:
"""
Description:
    A pole is attached by an un-actuated joint to a cart, which moves along
    a frictionless track. The pendulum starts upright, and the goal is to
    prevent it from falling over by increasing and reducing the cart's
    velocity.
Observation:
    Type: Box(4)
    Num     Observation               Min                     Max
    0       Cart Position             -4.8                    4.8
    1       Cart Velocity             -Inf                    Inf
    2       Pole Angle                -0.418 rad (-24 deg)    0.418 rad (24 deg)
    3       Pole Angular Velocity     -Inf                    Inf
Actions:
    Type: Discrete(2)
    Num   Action
    0     Push cart to the left
    1     Push cart to the right
    Note: The amount the velocity that is reduced or increased is not
    fixed; it depends on the angle the pole is pointing. This is because
    the center of gravity of the pole increases the amount of energy needed
    to move the cart underneath it
Reward:
    Reward is 1 for every step taken, including the termination step
Starting State:
    All observations are assigned a uniform random value in [-0.05..0.05]
Episode Termination:
    Pole Angle is more than 12 degrees.
    Cart Position is more than 2.4 (center of the cart reaches the edge of
    the display).
    Episode length is greater than 200.
    Solved Requirements:
    Considered solved when the average return is greater than or equal to
    195.0 over 100 consecutive trials.
"""


env = gym.make("CartPole-v1")

In [ ]:
# quick experiment to see what the maximum velocities are that the environment really returns
cart_vs = []
pole_vs = []
for i in range(3000):
    cart_p, cart_v, pole_a, pole_v = env.reset()
    cart_vs.append(cart_v)
    pole_vs.append(pole_v)
    terminal = False
    while not terminal:
        (cart_p, cart_v, pole_a, pole_v), reward, terminal, _ = env.step(env.action_space.sample())
        cart_vs.append(cart_v)
        pole_vs.append(pole_v)
min(cart_vs), max(cart_vs), min(pole_vs), max(pole_vs)

# Agent

$\pi(A_{t}=a|S_{t}=s,\theta) = \theta^{T} * x(s)$

In [ ]:
class Agent:
    
    def __init__(self, env, alpha_critic=0.01, alpha_actor=0.01, alpha_avg_reward=0.01, num_tilings=8, num_tiles=8, iht_size=4096):
        self.action_space = env.action_space
        self.actions = list(range(self.action_space.n))
        self.alpha_avg_reward = alpha_avg_reward
        self.alpha_critic = alpha_critic
        self.alpha_actor = alpha_actor
        
        self.last_action = None
        self.last_softmax_probs = None
        self.last_state = None
        self.last_features = None
        
        self.iht_size = iht_size
        self.num_tilings = num_tilings
        self.num_tiles = num_tiles
        self.iht = tc.IHT(iht_size)
        
        self.critic_w = np.ones(self.iht_size)
        self.actor_w = [np.zeros(self.iht_size) for i in range(self.action_space.n)]
        self.avg_reward = 0.
    
    def state_to_tiles(self, state):
        cart_pos, cart_vel, pole_ang, pole_vel = state
        cart_pos_scaled = ((cart_pos + 4.8) / 9.6) * self.num_tiles
        cart_vel_scaled = ((cart_vel + 5) / 10) * self.num_tiles  # assuming max cart velocity of 5
        pole_ang_scaled = ((pole_ang + 0.418) / 0.836) * self.num_tiles
        pole_vel_scaled = ((pole_vel + 5) / 10) * self.num_tiles  # assuming max pole velocity of 5
        return np.array(tc.tiles(self.iht, self.num_tilings, [cart_pos_scaled, cart_vel_scaled, pole_ang_scaled, pole_vel_scaled]))
    
    @staticmethod
    def softmax_stable(logits):
        z = logits - max(logits)
        num = np.exp(z)
        den = np.sum(num)
        return num / den
    
    def select_action(self, softmax_probs):
        action = np.random.choice(self.actions, p=softmax_probs)
        return action
    
    def agent_start(self, state):
        features = self.state_to_tiles(state)
        action_logits = self.get_all_actor_logits(features)
        softmax_probs = self.softmax_stable(action_logits)
        self.last_action = self.select_action(softmax_probs)
        self.last_softmax_probs = softmax_probs
        self.last_state = state
        self.last_features = features
        return self.last_action
    
    def agent_step(self, reward, state):
        features = self.state_to_tiles(state)
        action_logits = self.get_all_actor_logits(features)
        softmax_probs = self.softmax_stable(action_logits)
        action = self.select_action(softmax_probs)
        # calculate delta
        delta = reward - self.avg_reward + self.get_critic_estimate(features) - self.get_critic_estimate(self.last_features)
        # update average reward estimate
        self.avg_reward += self.alpha_avg_reward * delta
        # update critic weights
        self.critic_w[self.last_features] += self.alpha_critic * delta  # *1 for the gradient officially
        # update actor weights
        self.actor_w[self.last_action][self.last_features] += self.alpha_actor * delta * (1 - self.last_softmax_probs[self.last_action])  # selected action
        other_action = (1 - self.last_action)
        self.actor_w[other_action][self.last_features] += self.alpha_actor * delta * (0 - self.last_softmax_probs[other_action])  # other action
        # select next action
        self.last_action = action
        self.last_softmax_probs = softmax_probs
        self.last_state = state
        self.last_features = features
        
    def get_critic_estimate(self, features):
        return self.critic_w[features].sum()
    
    def get_actor_logit(self, features, action):
        return self.actor_w[action][features].sum()
    
    def get_all_actor_logits(self, features):
        action_values = []
        for a in range(self.action_space.n):
            action_values.append(self.get_actor_logit(features, a))
        return np.array(action_values)

# Helper functions

In [ ]:
def episode(env, agent):
    """Run one (training) episode.
    """
    last_observation = env.reset()
    terminal = False
    cumulative_reward = 0
    state_list = []
    agent.agent_start(last_observation)
    while not terminal:
        observation, reward, terminal, info = env.step(agent.last_action)
        agent.agent_step(reward, observation)
        cumulative_reward += reward
        state_list.append(observation)
    return cumulative_reward, state_list


def run_experiment(env, agent, n_episodes=1000):
    reward_list = []
    state_lists = []
    for i in tqdm(range(n_episodes)):
        episode_reward, state_list = episode(env, agent)
        reward_list.append(episode_reward)
        state_lists.append(state_list)
    return reward_list, state_lists


def animate_episode(env, agent):
    if agent == 'random':
        action_fnc = lambda x: env.action_space.sample()
    elif agent == 'right':
        action_fnc = lambda x: 2
    else:
        action_fnc = lambda x: agent.select_action(agent.state_to_tiles(x))[0]
    obs = env.reset()
    terminal = False
    i = 0
    while not terminal and i < 500:
        obs, _, terminal, _ = env.step(action_fnc(obs))
        env.render()
        i += 1
        time.sleep(1/30)
    env.close()

# Example episodes

# Start the experiment

In [ ]:
agent = Agent(env, alpha_critic=0.0625, alpha_actor=0.0078, alpha_avg_reward=0.0156)

In [ ]:
rewards, observations = run_experiment(env, agent, n_episodes=2000)
observations = np.array([s for lst in observations for s in lst])

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plt.plot(pd.Series(rewards).rolling(window=50).mean())
ax.set_title("Rolling mean of total reward per episode", fontsize=20)
ax.set_xlabel("Episode Number", fontsize=16)
ax.set_ylabel("Reward per Episode", fontsize=16);

In [ ]:
states = pd.DataFrame(observations[:,[0,2]], columns=['cart_pos', 'pole_ang'])
cart_pos_linspace = np.linspace(-4.8, 4.8, 97)
pole_ang_linspace = np.linspace(-0.418, 0.418, 97)
states['cart_pos_bin'] = pd.IntervalIndex(pd.cut(states['cart_pos'], cart_pos_linspace)).mid
states['pole_ang_bin'] = pd.IntervalIndex(pd.cut(states['pole_ang'], pole_ang_linspace)).mid
state_visitation_counts = states.groupby(['cart_pos_bin', 'pole_ang_bin'])['cart_pos'].count().unstack(fill_value=0)
state_visitation_counts.index = np.round(state_visitation_counts.index, 3)
state_visitation_counts.columns = np.round(state_visitation_counts.columns, 3)

In [ ]:
# create heatmap of visitation count
fig, ax = plt.subplots(figsize=(18, 9))
sns.heatmap(np.log(state_visitation_counts + 1).T.sort_index(ascending=False), ax=ax)
ax.set_title("Natural log state visitation counts", fontsize=20)
ax.set_xlabel("cart position", fontsize=16)
ax.set_ylabel("pole angle", fontsize=16);

In [ ]:
# create data frames with state values and top action
cart_pos_linspace = np.linspace(-4.8, 4.8, 97)
pole_ang_linspace = np.linspace(-0.418, 0.418, 97)
df_state_values = pd.DataFrame(index=cart_pos_linspace, columns=pole_ang_linspace, dtype='float32')
for i in cart_pos_linspace:
    for j in pole_ang_linspace:
        features = agent.state_to_tiles(np.array([i,0,j,0]))
        df_state_values.loc[i, j] = agent.get_critic_estimate(features)
df_state_values.index = np.round(df_state_values.index, 3)
df_state_values.columns = np.round(df_state_values.columns, 3)

In [ ]:
# create heatmaps of state values and top action
fig, ax = plt.subplots(figsize=(18, 9))
sns.heatmap(df_state_values.T.sort_index(ascending=False), ax=ax)
ax.set_title("State value estimates", fontsize=20);